In [131]:
from pymongo import MongoClient
import pandas as pd
import folium

In [132]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [133]:
db, coll = connectCollection('companies','companies')

In [134]:
companies=list(coll.find())

df=pd.DataFrame(companies)

In [135]:
print(set(df.category_code))

{'security', 'travel', 'semiconductor', 'software', 'automotive', 'analytics', 'real_estate', 'enterprise', 'other', 'nanotech', 'social', 'web', 'photo_video', 'health', 'biotech', 'design', 'education', 'cleantech', 'public_relations', 'network_hosting', 'music', 'legal', 'news', 'transportation', 'local', 'finance', 'mobile', 'hospitality', 'government', 'search', 'medical', 'messaging', None, 'fashion', 'nonprofit', 'sports', 'games_video', 'hardware', 'manufacturing', 'advertising', 'consulting', 'ecommerce'}


In [136]:
tech_companies=list(coll.find({"$and" : [{"$or":[{"category_code": 'web'},{'category_code':'analytics'},
                                                 {"category_code": 'software'},{"category_code": 'hardware'}]}, 
                                         {"offices.city": "New York"}]}))
companies_df=pd.DataFrame(tech_companies)

In [137]:
def getLocation(comp):
    longitude = comp['offices'][0]['longitude']
    latitude = comp['offices'][0]['latitude']
    loc = {
        'type':'Point',
        'coordinates':[latitude, longitude]
    }
    return loc

In [138]:
for e in tech_companies:
    value={"$set" : {"loc": getLocation(e)}}
    coll.update_one(e,value)

In [165]:
db, coll = connectCollection('companies','companies')

In [166]:
companies_df=pd.DataFrame(tech_companies)

print(companies_df['loc'])

companies_df['loc']

0      {'type': 'Point', 'coordinates': [47.603122, -...
1      {'type': 'Point', 'coordinates': [40.757725, -...
2      {'type': 'Point', 'coordinates': [40.72604, -7...
3      {'type': 'Point', 'coordinates': [37.393679, -...
4      {'type': 'Point', 'coordinates': [40.736416, -...
                             ...                        
296    {'type': 'Point', 'coordinates': [40.751804, -...
297       {'type': 'Point', 'coordinates': [None, None]}
298    {'type': 'Point', 'coordinates': [40.72616, -7...
299    {'type': 'Point', 'coordinates': [40.72616, -7...
300       {'type': 'Point', 'coordinates': [None, None]}
Name: loc, Length: 301, dtype: object


0      {'type': 'Point', 'coordinates': [47.603122, -...
1      {'type': 'Point', 'coordinates': [40.757725, -...
2      {'type': 'Point', 'coordinates': [40.72604, -7...
3      {'type': 'Point', 'coordinates': [37.393679, -...
4      {'type': 'Point', 'coordinates': [40.736416, -...
                             ...                        
296    {'type': 'Point', 'coordinates': [40.751804, -...
297       {'type': 'Point', 'coordinates': [None, None]}
298    {'type': 'Point', 'coordinates': [40.72616, -7...
299    {'type': 'Point', 'coordinates': [40.72616, -7...
300       {'type': 'Point', 'coordinates': [None, None]}
Name: loc, Length: 301, dtype: object

In [167]:
companies_df=companies_df[['name','loc']]

companies_df

,name,loc
0,Wetpaint,"{'type': 'Point', 'coordinates': [47.603122, -..."
1,Viacom,"{'type': 'Point', 'coordinates': [40.757725, -..."
2,Meetup,"{'type': 'Point', 'coordinates': [40.72604, -7..."
3,Meebo,"{'type': 'Point', 'coordinates': [37.393679, -..."
4,Forbes,"{'type': 'Point', 'coordinates': [40.736416, -..."
...,...,...
296,Datum Legal,"{'type': 'Point', 'coordinates': [40.751804, -..."
297,MedTrackAlert,"{'type': 'Point', 'coordinates': [None, None]}"
298,Vertro,"{'type': 'Point', 'coordinates': [40.72616, -7..."
299,ALOT,"{'type': 'Point', 'coordinates': [40.72616, -7..."


In [128]:
for i in range(len(companies_df['loc'])):
    if companies_df.loc[i]['loc']==[None,None]:
        print(True)
        break

True


In [168]:
companies_df=companies_df.loc[companies_df['loc']!={'type': 'Point', 'coordinates': [None, None]}]

companies_df

,name,loc
0,Wetpaint,"{'type': 'Point', 'coordinates': [47.603122, -..."
1,Viacom,"{'type': 'Point', 'coordinates': [40.757725, -..."
2,Meetup,"{'type': 'Point', 'coordinates': [40.72604, -7..."
3,Meebo,"{'type': 'Point', 'coordinates': [37.393679, -..."
4,Forbes,"{'type': 'Point', 'coordinates': [40.736416, -..."
...,...,...
294,Sword,"{'type': 'Point', 'coordinates': [40.7029757, ..."
295,smartFOCUS,"{'type': 'Point', 'coordinates': [51.4525116, ..."
296,Datum Legal,"{'type': 'Point', 'coordinates': [40.751804, -..."
298,Vertro,"{'type': 'Point', 'coordinates': [40.72616, -7..."


In [14]:
#para crear nuevas collections
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["Finding_best_place"]
#para importar el contenido de un csv a mongo
#mongoimport --db companies --collection finding_location --type csv --file Companies.csv --headerline

In [169]:
companies_df=companies_df.loc[:100]

In [164]:
companies_df['Category']='Company'

companies_df=companies_df[['name','loc','Category']]

companies_df[['name','loc','Category']].to_json('companies.json',orient='records')

In [161]:
for i in companies_df.index:
    if type(companies_df.loc[i]['loc']['coordinates'][0])!=float:
        print(companies_df.loc[i])

In [160]:
companies_df.index

Int64Index([  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,
            ...
            283, 285, 288, 289, 293, 294, 295, 296, 298, 299],
           dtype='int64', length=235)

In [ ]:
map_city = folium.Map(location=(40.7142700, -74.0059700), zoom_start=12)
map_city

In [ ]:
for e in coordinates_comp:
    folium.Marker(e, radius=2,
                    icon=folium.Icon(prefix='fa',icon='cloud',color='red'), 
                   ).add_to(map_city)

In [ ]:
map_city

In [33]:
companies_df[:12]

,_id,name,loc,Category
0,52cdef7c4bab8bd675297d8a,Wetpaint,"{'type': 'Point', 'coordinates': [-122.333253,...",Company
1,52cdef7c4bab8bd675297d9f,Viacom,"{'type': 'Point', 'coordinates': [-73.986011, ...",Company
2,52cdef7c4bab8bd675297db2,Meetup,"{'type': 'Point', 'coordinates': [-73.995722, ...",Company
3,52cdef7c4bab8bd675297e09,Meebo,"{'type': 'Point', 'coordinates': [-122.079017,...",Company
4,52cdef7c4bab8bd675297e71,Forbes,"{'type': 'Point', 'coordinates': [-73.993302, ...",Company
5,52cdef7c4bab8bd675297e96,AOL,"{'type': 'Point', 'coordinates': [-73.9919311,...",Company
7,52cdef7c4bab8bd675297ee2,Veotag,"{'type': 'Point', 'coordinates': [-73.98546, 4...",Company
8,52cdef7c4bab8bd675297ef2,ContextWeb,"{'type': 'Point', 'coordinates': [-74.010808, ...",Company
9,52cdef7c4bab8bd675297f14,Heavy,"{'type': 'Point', 'coordinates': [-73.98962, 4...",Company
10,52cdef7c4bab8bd675297f16,Buzzmetrics,"{'type': 'Point', 'coordinates': [-73.991933, ...",Company
